In [ ]:
!pip install langchain_community langchainhub chromadb langchain langchain-openai

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_API_KEY')

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_paths=["https://www.educosys.com/course/genai", "https://www.educosys.com/#all_courses","https://www.educosys.com/#faq"])

docs = loader.load()

print(docs)

[Document(metadata={'source': 'https://www.educosys.com/course/genai', 'title': 'Hands-on Generative AI Course', 'description': 'Learn, Build, Deploy and Apply Generative AI', 'language': 'en'}, page_content="Hands-on Generative AI CourseCoursesBundle CoursesUpcoming CoursesMentorFree ContentTestimonialsFAQLogin Signup Ongoing LIVE CourseHands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AIJoin Anytime!Get LifeTime AccessAccess all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics Architectures: Feedforward, RNN, and CNN Mini Project - Build a Simple Neural Network Using TensorFlow Mini Project - Train an Autoencoder on the MNIST Dataset2Week 2Deep Generative Models Discriminative and Generative mode

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)

In [ ]:
print(splits[0])
print(splits[1])
print(splits[2])

page_content='Hands-on Generative AI CourseCoursesBundle CoursesUpcoming CoursesMentorFree ContentTestimonialsFAQLogin Signup Ongoing LIVE CourseHands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AIJoin Anytime!Get LifeTime AccessAccess all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics Architectures: Feedforward, RNN, and CNN Mini Project - Build a Simple Neural Network Using TensorFlow Mini Project - Train an Autoencoder on the MNIST Dataset2Week 2Deep Generative Models Discriminative and Generative models Generative Adversarial Networks (GANs) Variational Autoencoders (VAEs) Probabilistic Data Generation Using VAEs Four Mini Projects using TensorFlow Metrics Visualization using TensorBoard Mini

In [ ]:
print(len(splits))

19


In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
print(vectorstore._collection.count())

In [ ]:
print(vectorstore._collection.get())

In [ ]:
print("\nCollection 1 - ", vectorstore._collection.get(ids=['28651d9a-ab51-41f8-ab83-e68285623c4e'], include=["embeddings", "documents"]))
print("\nCollection 2 - ", vectorstore._collection.get(ids=['054dee19-19ed-4574-bc51-511060fd707a'], include=["embeddings", "documents"]))
print("\nCollection 3 - ", vectorstore._collection.get(ids=['2fd71cb4-835a-43c5-b920-b7e1be51c450'], include=["embeddings", "documents"]))

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(docs):
  return "\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = ({"context" : retriever | format_docs, "question" : RunnablePassthrough()}
             | prompt
             | llm
             | StrOutputParser())

In [ ]:
rag_chain.invoke("Are the recordings of the course available? For how long?")

In [ ]:
rag_chain.invoke("Are the testimonials for the course available? Name the studenst who have shared testimonials")

In [ ]:
rag_chain.invoke("Are the certificates for the course provided?")

In [ ]:
rag_chain.invoke("What all projects are covered in the course?")

In [ ]:
from langchain_core.runnables import RunnableLambda

In [ ]:
def print_prompt(prompt_text):
  print("Prompt - ", prompt_text)
  return prompt_text

In [ ]:
rag_chain_with_print = ({"context" : retriever | format_docs, "question" : RunnablePassthrough()}
             | prompt
             | RunnableLambda(print_prompt)
             | llm
             | StrOutputParser())

In [ ]:
rag_chain_with_print.invoke("What all projects are covered in the course?")